In [ ]:
''' Connecting Google Drive File Directory '''

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install mysql-connector-python
%pip install google-cloud-bigquery
%pip install pandas
%pip install pyarrow

In [ ]:
''' Importing required Libraries '''

import pandas as pd
import mysql.connector as connection
from google.cloud import bigquery
import os

In [ ]:
'''
  This block of code prompts the user to input the credentials required for the database connection
  The user input answers are appended to the List called userinput
'''
userinput = []
textout = 0

while textout <= 3:
  if textout == 0:
    inputhost = input('Enter your Host IP: ')
    userinput.append(inputhost)
    textout = textout + 1
  elif textout == 1:
    inputuser = input('Enter your DB Username: ')
    userinput.append(inputuser)
    textout = textout + 1
  elif textout == 2:
    inputpasswd = input('Enter your DB password: ')
    userinput.append(inputpasswd)
    textout = textout + 1
  elif textout == 3:
    inputdatabase = input('Enter your DB Name: ')
    userinput.append(inputdatabase)
    break # This allows to cut off the While Loop instead of increasing by one, one more time the texout variable. This prevents for an unecessary iteration to happen

#print(userinput[0])

In [ ]:
''' SQL DATABASE Credentials Connection '''

mydb = connection.connect(
    host=userinput[0],
    user=userinput[1],
    passwd=userinput[2],
    database=userinput[3]
)

In [ ]:
''' Obtaining the Tables Available in the Retail_DB Database '''

sql = 'SHOW TABLES'
df_tablenames = pd.read_sql(sql, mydb)
mydb.close()
list_tablenames = df_tablenames['Tables_in_retail_db'].tolist() # Passing the Table names to the list_tablenames variables which is a List
df_tablenames.head()
#type(df_tablenames)
#list_tablenames
#type(list_tablenames)

<ipython-input-135-6b330d684dee>:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tablenames = pd.read_sql(sql, mydb)


,Tables_in_retail_db
0,categories
1,customers
2,departments
3,order_items
4,orders


In [ ]:
'''
  This block of code prompts the user to Decide which of the obtained Tables,
  they want to migrate over to BigQuery

  The chooseinput variables will contain the list of Table names that will be migrated.
'''

print(f"The Available Tables are: {list_tablenames}")
print('You will Choose the Names of the Tables that you would like to migrate')

chooseinput = []
sizelistablenames = len(list_tablenames)
yes = 'Yes'

for i in range(sizelistablenames):
  decision = input(f"Would you like to migrate the {list_tablenames[i]} Table? -- Please Input: Yes or No: ")

  if decision.lower() == yes.lower():
    chooseinput.append(list_tablenames[i])
  else:
    continue

print(f"You have chosen to Migrate the following Tables: {chooseinput}")

The Available Tables are: ['categories', 'customers', 'departments', 'order_items', 'orders', 'products']
You will Choose the Names of the Tables that you would like to migrate
Would you like to migrate the categories Table? -- Please Input: Yes or No: Yes
Would you like to migrate the customers Table? -- Please Input: Yes or No: No
Would you like to migrate the departments Table? -- Please Input: Yes or No: No
Would you like to migrate the order_items Table? -- Please Input: Yes or No: Yes
Would you like to migrate the orders Table? -- Please Input: Yes or No: Yes
Would you like to migrate the products Table? -- Please Input: Yes or No: Yes
You have chosen to Migrate the following Tables: ['categories', 'order_items', 'orders', 'products']


In [ ]:
''' Configuring Google Cloud Credentials '''

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/MyDrive/ColabNotebooks/service.json' #json file is obtained from the Google Console Service Account Creation

In [ ]:
''' Creating a BigQuery Client '''

client = bigquery.Client()

In [ ]:
# Global Variables to be used
str_tableid = 'ccocapythonproject.hn_dp_dtlk_raw.' #used for creating the raw table id
str_initqry = 'SELECT * FROM retail_db.' #select statement used to create the different dataframes in iteration SELECT * FROM retail_db.TableName
size1 = len(chooseinput) #the Length of chooseinput determines how many times the Loop will execute for
list_tableid = []
list_querystatement = []

'''
  This for Loop is designed to build and execute the parameters required to
  copy over to BigQuery every Table from the Retail_DB Database.

  The tables are copied one by one per each iteration.
'''
for i in range(size1):

  # Establishing connection with DB per iteration
  mydb = connection.connect(
    host=userinput[0],
    user=userinput[1],
    passwd=userinput[2],
    database=userinput[3]
  )

  # Building Query Statements assigned to a List
  str_sql2 = str_initqry + chooseinput[i]
  list_querystatement.append(str_sql2)
  # print(list_querystatement) -- validating

  # Creating String used for the raw table id
  concatstr = str_tableid + chooseinput[i]
  list_tableid.append(concatstr)

  # Creating dataframes per iteration // not forgetting to close connection per iteration
  sql2 = list_querystatement[i]
  df_retaildb = pd.read_sql(sql2, mydb)
  mydb.close()
  # df_retaildb.head() -- validating

  # WRITING THE DATA-FRAME DATA TO GOOGLE BIGQUERY
  tables_property = client.get_table(list_tableid[i]) # API Request

  job_configuration = bigquery.LoadJobConfig(
      schema = tables_property.schema,
      write_disposition='WRITE_TRUNCATE' # This allows us to overwrite any existing data on the table if the table exists
  )

  # Load contents of a pandas DataFrame to a Table
  job = client.load_table_from_dataframe(
    df_retaildb, list_tableid[i], job_config=job_configuration
  )

  job.result() # Wait for the job to complete
  print("Upload Complete")



<ipython-input-139-4ac865c25877>:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_retaildb = pd.read_sql(sql2, mydb)


Upload Complete


<ipython-input-139-4ac865c25877>:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_retaildb = pd.read_sql(sql2, mydb)


Upload Complete


<ipython-input-139-4ac865c25877>:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_retaildb = pd.read_sql(sql2, mydb)


Upload Complete


<ipython-input-139-4ac865c25877>:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_retaildb = pd.read_sql(sql2, mydb)


Upload Complete
